# Importing libraries

In [1]:
!pip install pywsd
!pip install fuzzywuzzy
!pip install python-Levenshtein


import spacy 
sp = spacy.load('en_core_web_sm')

import csv
import pandas as pd
import numpy as np
import statistics

import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import wordnet

from sklearn.metrics import jaccard_score
from pywsd.utils import lemmatize_sentence




     |████████████████████████████████| 26.8MB 162kB/s 
     |████████████████████████████████| 31.6MB 132kB/s 
  Created wheel for pywsd: filename=pywsd-1.2.4-cp36-none-any.whl size=26940455 sha256=8825008ae014d3d43138d72801718e9a0e7a934dc05cbf7f9a68cebe3a48a3d3
  Stored in directory: /root/.cache/pip/wheels/71/4d/d2/405b948047f7f3851f16ab9d893ce7c1a3010182900884536b
  Created wheel for wn: filename=wn-0.0.23-cp36-none-any.whl size=31792944 sha256=8d499cbd82ca16dfe07e1be76afcb68012c04d3c76b2ab1ab5510455f88d42ee
  Stored in directory: /root/.cache/pip/wheels/56/e3/c4/886021dbf4d758dc3cb9ddaa47d7d6fc895240d83f010e6305
Successfully built pywsd wn
     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144791 sha256=bc015299b116042ff0ef889a82e6d008bf2b78c97f38dd65bc38850fa7c601e6
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342

Warming up PyWSD (takes ~10 secs)... took 5.294661045074463 secs.


# Loading the dataset from Microsoft Research Paraphrase Corpus

In [2]:
from google.colab import files
uploaded = files.upload()

Saving msr_paraphrase_train.txt to msr_paraphrase_train.txt


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['msr_paraphrase_train.txt']), sep='\t', quoting=csv.QUOTE_NONE)
df

,Quality,#1 ID,#2 ID,#1 String,#2 String
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi..."
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an..."
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ..."
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...
...,...,...,...,...,...
4071,1,1620264,1620507,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull..."
4072,0,1848001,1848224,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...
4073,1,747160,747144,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p..."
4074,1,2539933,2539850,The notification was first reported Friday by ...,MSNBC.com first reported the CIA request on Fr...


# Experimenting with the Jaccard Score

In [65]:
# def calculate_jaccard_score(sen1, sen2):
#   array1 = lemmatize_sentence(sen1) 
#   array2 = lemmatize_sentence(sen2)

#   if(len(array1) < len(array2)):
#     padded = np.append(array1, [""]*(len(array2)-len(array1)))
#     score = jaccard_score(padded, array2, average="macro")

#   elif(len(array1) > len(array2)):
#       padded = np.append(array2, [""]*(len(array1)-len(array2)))
#       score = jaccard_score(padded, array1, average="macro")
#   else: 
#       score = jaccard_score(array1, array2, average="macro")
#   return score



In [90]:
def calculate_jaccard_score(sen1, sen2):
  array1 = set(lemmatize_sentence(sen1) )
  array2 = set(lemmatize_sentence(sen2))

  intersection = array1.intersection(array2)
  union = array1.union(array2)
        
  return float(len(intersection)) / len(union)

In [91]:
df["Score_jaccard"] = df.apply(lambda row: calculate_jaccard_score(row['#1 String'], row['#2 String']), axis = 1)


In [101]:
df["result_jaccard"] = df['Score_jaccard'] >= 0.4
df["Right_Results_jaccard"] = df['result_jaccard'] == df['Quality'] 
df.head(10)

,Quality,#1 ID,#2 ID,#1 String,#2 String,Score_jaccard,result_jaccard,Right_Results_jaccard,Score_fuzzy,result_fuzzy,Right_Results_fuzzy,Score,Score_TF,Right_Results,result_wordnet,Right_Results_wordnet
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",0.636364,True,True,True,True,True,True,True,True,True,True
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0.500000,True,False,False,False,True,False,False,True,False,True
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",0.636364,True,True,True,True,True,True,True,True,True,True
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0.466667,True,False,True,True,False,True,True,False,True,False
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,0.413793,True,True,False,False,False,True,True,True,True,True
5,1,738533,737951,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,0.625000,True,True,True,True,True,True,True,True,True,True
6,0,264589,264502,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0.269231,False,True,False,False,True,False,False,True,False,True
7,1,579975,579810,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,0.428571,True,True,True,True,True,False,False,False,False,False
8,0,3114205,3114194,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0.269231,False,True,False,False,True,False,False,True,False,True
9,1,1355540,1355592,He said the foodservice pie business doesn't f...,"""The foodservice pie business does not fit our...",0.555556,True,True,True,True,True,True,True,True,True,True


Jaccard score measures the similarity of two sentences. It is defined as the size of the intersection divided by the size of the union of two sentences. [[1]](https://neo4j.com/docs/graph-algorithms/current/labs-algorithms/jaccard/#:~:text=Jaccard%20Similarity%20(coefficient)%2C%20a,the%20union%20of%20two%20sets.)

In the calculate_jaccard_score() method, we have two sentences as our inputs. We first lemmatize all the words in those sentences for comparison. Then we calculate the jaccard_score by definition. 
In the final step, we just put a threshhold to determine whether the results generated by this method are true or false. In my code I used 0.4 as my threshold, which was chosen after some adjustments. 

I attempted to use sklearn jaccard_score, but after implementing the results were far from being good. The code is commented in the first cell and below you can see my explanation. 

In the calculate_jaccard_score() method, we have two sentences as our inputs. We first lemmatize all the words in those sentences for comparison. then we use padding to have two arrays of similar sizes( while experimenting with different strings ), after which we use the built-in jaccard_score() method to calculate the score. In the final step, we just put a threshhold to determine whether the results generated by this method are true or false. 

In the final step, we just put a threshold to determine whether the results generated by this method are true or false. In my code I used 0.01 as my threshold, which was chosen after some adjustments.  


In [102]:
df.Right_Results_jaccard.value_counts()

True     2978
False    1098
Name: Right_Results_jaccard, dtype: int64

In [103]:
df['Right_Results_jaccard'].value_counts(normalize=True).mul(100).astype(str)+'%'

True      73.06182531894014%
False    26.938174681059863%
Name: Right_Results_jaccard, dtype: object

# Solution with fuzzywuzzy library

In [33]:
from fuzzywuzzy import fuzz

def full_or_almost_full_plagiarism_check(Sentence_one, Sentence_two):
    Token_Set_Ratio = fuzz.token_set_ratio(Sentence_one, Sentence_two)
   # if (len(Sentence_one) - len(Sentence_two) > 10):
   #    return False
    if (Token_Set_Ratio) >= 80:
        return True
    return False
    

In [34]:
df["Score_fuzzy"] = df.apply(lambda row: full_or_almost_full_plagiarism_check(row['#1 String'], row['#2 String']), axis = 1)
df["Right_Results_fuzzy"] = df['result_fuzzy'] == df['Quality'] 
df

,Quality,#1 ID,#2 ID,#1 String,#2 String,Score_jaccard,result_jaccard,Right_Results_jaccard,Score_fuzzy,result_fuzzy,Right_Results_fuzzy
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",0.000000,False,False,True,True,True
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0.136364,True,False,False,False,True
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",0.022727,True,True,True,True,True
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0.000000,False,True,True,True,False
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,0.041667,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
4071,1,1620264,1620507,"""At this point, Mr. Brando announced: 'Somebod...","Brando said that ""somebody ought to put a bull...",0.000000,False,False,True,True,True
4072,0,1848001,1848224,"Martin, 58, will be freed today after serving ...",Martin served two thirds of a five-year senten...,0.041667,True,False,True,True,False
4073,1,747160,747144,"""We have concluded that the outlook for price ...","In a statement, the ECB said the outlook for p...",0.000000,False,False,True,True,True
4074,1,2539933,2539850,The notification was first reported Friday by ...,MSNBC.com first reported the CIA request on Fr...,0.111111,True,True,False,False,False


In [35]:
df.Right_Results_fuzzy.value_counts()

True     2850
False    1226
Name: Right_Results_fuzzy, dtype: int64

Fuzzy-Wuzzy predicts correct results for 70% of our data.

# Algorithm using WordNet


Inspiration for this approach was taken from the following tutorial: https://nlpforhackers.io/wordnet-sentence-similarity/

Idea for the algorithm is as follow:
We split the senteces into separate words and wrap the words into synsets from the WordNet library. Separate synsets can be compared. Similar synsets result in a higher score.
We need to devise a method for comparing the whole sentences. There are a lot of ways. In this algorithm every word from the first sentence is compared with every word from the second sentence. The words with maximum similarity are chosen and avarage is calculate.



In [62]:
def tokenize_words(sentence):
    tokens = nltk.word_tokenize(sentence) 
    return tokens

In [63]:
def get_synsets(words):
  synsets=[]
  for word in words:
    if(len(wordnet.synsets(word))>0):
      synsets.append(wordnet.synsets(word)[0])

  return synsets


In [86]:
print_scores=True

def calculate_similarity_score_with_synonyms_not_symmetric(sen1,sen2):
  words1=tokenize_words(sen1)
  words2=tokenize_words(sen2)

  synsets1 = get_synsets(words1) 
  synsets2 = get_synsets(words2)

  similarities = []
  for synset_from_1 in synsets1:
    array_of_similarities = [synset_from_2.wup_similarity(synset_from_1) for synset_from_2 in synsets2 if synset_from_2.wup_similarity(synset_from_1) is not None]
    if(len(array_of_similarities)>0):
      max_similarity = max(array_of_similarities)
      similarities.append(max_similarity)

  if (len(similarities) == 0):
    return 0
  mean=statistics.mean(similarities)
    
  return mean

def calculate_similarity_score_with_synonyms_wordnet(sen1,sen2):
  score = (calculate_similarity_score_with_synonyms_not_symmetric(sen1,sen2) + calculate_similarity_score_with_synonyms_not_symmetric(sen2,sen1))/2
  if(print_scores):
    print("")
    print(score)
  return score >= 0.736


Then I checked this algorithm for smaller examples and noticed that it behaves reasonably for them.

In [67]:
print(calculate_similarity_score_with_synonyms("I am a feline","I am a cat"))
print(calculate_similarity_score_with_synonyms("cat dog moon","play"))
print(calculate_similarity_score_with_synonyms("cat","cat"))
print(calculate_similarity_score_with_synonyms("woman is beautiful and elegant","girl is pretty"))
print(calculate_similarity_score_with_synonyms("woman is beautiful","woman is pretty"))
print(calculate_similarity_score_with_synonyms("children play football", "tiger eats its pray"))
print(calculate_similarity_score_with_synonyms("Her life spanned years of incredible change for women as they gained more rights than ever before.","She lived through the exciting era of women's liberation."))
print(calculate_similarity_score_with_synonyms("Armenia is a beautiful country and it is one of the most mountainous countries in the world","Armenia is a beautiful country"))


0.9907407407407407
True

0.1255952380952381
False

1.0
True

0.736842105263158
True

0.8333333333333334
True

0.43350599349051366
False

0.5076120259169794
False

0.8811651583710407
True


The threshold was chosen experimentally to give results as good as possible. There are many ways of potentally improving the algorithm. We can choose a different similarity measure (instead of the maximum and the avarage). We can also extract only nouns and verbs and put only them to the wordnet library. (The later approach was attempted but it didn't give better results on the analyzed dataset.)


In [106]:
print_scores=False

df["result_wordnet"] = df.apply(lambda row: calculate_similarity_score_with_synonyms_wordnet(row['#1 String'], row['#2 String']), axis = 1)
df["Right_Results_wordnet"] = df['result_wordnet'] == df['Quality'] 
df.head(10)

,Quality,#1 ID,#2 ID,#1 String,#2 String,Score_jaccard,result_jaccard,Right_Results_jaccard,Score_fuzzy,result_fuzzy,Right_Results_fuzzy,Score,Score_TF,Right_Results,result_wordnet,Right_Results_wordnet
0,1,702876,702977,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",0.636364,True,True,True,True,True,True,True,True,True,True
1,0,2108705,2108831,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,0.500000,True,False,False,False,True,False,False,True,False,True
2,1,1330381,1330521,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",0.636364,True,True,True,True,True,True,True,True,True,True
3,0,3344667,3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",0.466667,True,False,True,True,False,True,True,False,True,False
4,1,1236820,1236712,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,0.413793,True,True,False,False,False,True,True,True,True,True
5,1,738533,737951,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,0.625000,True,True,True,True,True,True,True,True,True,True
6,0,264589,264502,"The Nasdaq had a weekly gain of 17.27, or 1.2 ...",The tech-laced Nasdaq Composite .IXIC rallied ...,0.269231,False,True,False,False,True,False,False,True,False,True
7,1,579975,579810,The DVD-CCA then appealed to the state Supreme...,The DVD CCA appealed that decision to the U.S....,0.428571,True,True,True,True,True,False,False,False,False,False
8,0,3114205,3114194,"That compared with $35.18 million, or 24 cents...",Earnings were affected by a non-recurring $8 m...,0.269231,False,True,False,False,True,False,False,True,False,True
9,1,1355540,1355592,He said the foodservice pie business doesn't f...,"""The foodservice pie business does not fit our...",0.555556,True,True,True,True,True,True,True,True,True,True


In [88]:
df.Right_Results.value_counts()

True     2876
False    1200
Name: Right_Results, dtype: int64

In [105]:
df.head(10)
df[['Quality', '#1 String', "#2 String", "result_jaccard", "Right_Results_jaccard", "result_fuzzy", "Right_Results_fuzzy", "result_wordnet", "Right_Results_wordnet"]].head(6)

,Quality,#1 String,#2 String,result_jaccard,Right_Results_jaccard,result_fuzzy,Right_Results_fuzzy,result_wordnet,Right_Results_wordnet
0,1,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...",True,True,True,True,True,True
1,0,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,True,False,False,True,False,True
2,1,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...",True,True,True,True,True,True
3,0,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...",True,False,True,False,True,False
4,1,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,True,True,False,False,True,True
5,1,Revenue in the first quarter of the year dropp...,With the scandal hanging over Stewart's compan...,True,True,True,True,True,True


This algorithms predicts correct results for 70.5% of our data.

# Validation

We want to check if our algorithms behave any better than just stating that every sentence in the dataset is a plagiarism. 


In [72]:
df.Quality.value_counts()

1    2753
0    1323
Name: Quality, dtype: int64

Apparently the three presented algorithms did not perform much better than just stating that every 2 sentences are a plagiarism. To assess their value one could try to run them on a different dataset with possible simpler paraphrases. The MSRP dataset contains very long and difficult sentences even for a human. Unfortunately, none of the presented algorithms understands the meaning of the presented sentences, so it cannot handle a true paraphrase where completely different words are used to describe the same idea.


It seems like the implementation of the jaccard_similarity by using its definition performs best, which to be honest surprised me, as I initally got much lower results. 